In [1]:
import argparse
import os
import torch
from libs.Loader import Dataset
from libs.Matrix import MulLayer
import torchvision.utils as vutils
import torch.backends.cudnn as cudnn
from libs.models import encoder3,encoder4
from libs.models import decoder3,decoder4

In [2]:
class MATparser:
    vgg_dir ='models/vgg_r41.pth'
    decoder_dir ='models/dec_r41.pth'
    matrixPath ='models/r41.pth'
    stylePath ="data/style/"
    contentPath ="data/MMMphotoes/"
    outf ="ArtisticMMMphotoes/"
    batchSize =1
    loadSize =512
    fineSize =512
    layer ="r41"
    cuda = True

In [3]:
################# PREPARATIONS #################
#opt = parser.parse_args()
#opt.cuda = torch.cuda.is_available()
opt = MATparser()
print(opt)
os.makedirs(opt.outf,exist_ok=True)
cudnn.benchmark = True

In [4]:
################# DATA #################
content_dataset = Dataset(opt.contentPath,opt.loadSize,opt.fineSize,test=True)

In [5]:
content_loader = torch.utils.data.DataLoader(dataset=content_dataset,
                                             batch_size = opt.batchSize,
                                             shuffle = False,
                                             num_workers = 1)

In [6]:
style_dataset = Dataset(opt.stylePath,opt.loadSize,opt.fineSize,test=True)
style_loader = torch.utils.data.DataLoader(dataset=style_dataset,
                                           batch_size = opt.batchSize,
                                           shuffle = False,
                                           num_workers = 1)

################# MODEL #################
if(opt.layer == 'r31'):
    matrix = MulLayer('r31')
    vgg = encoder3()
    dec = decoder3()
elif(opt.layer == 'r41'):
    matrix = MulLayer('r41')
    vgg = encoder4()
    dec = decoder4()
vgg.load_state_dict(torch.load(opt.vgg_dir))
dec.load_state_dict(torch.load(opt.decoder_dir))
matrix.load_state_dict(torch.load(opt.matrixPath))

for param in vgg.parameters():
    param.requires_grad = False
for param in matrix.parameters():
    param.requires_grad = False
for param in dec.parameters():
    param.requires_grad = False

################# GLOBAL VARIABLE #################
contentV = torch.Tensor(opt.batchSize,3,opt.fineSize,opt.fineSize)
styleV = torch.Tensor(opt.batchSize,3,opt.fineSize,opt.fineSize)

################# GPU  #################
if(opt.cuda):
    vgg.cuda()
    dec.cuda()
    matrix.cuda()
    contentV = contentV.cuda()
    styleV = styleV.cuda()

In [7]:
(content,contentName) = next(iter(content_loader))
#content
contentName

('fox1',)

In [15]:
# اضافه کردن واترمارک
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

for ci,(content,contentName) in enumerate(content_loader):
    contentName = contentName[0]
    contentV.data.resize_(content.size()).copy_(content)
    for sj,(style,styleName) in enumerate(style_loader):
        styleName = styleName[0]
        styleV.data.resize_(style.size()).copy_(style)

        # forward
        with torch.no_grad():
            sF = vgg(styleV)
            cF = vgg(contentV)

            if(opt.layer == 'r41'):
                feature,transmatrix = matrix(cF[opt.layer],sF[opt.layer])
            else:
                feature,transmatrix = matrix(cF,sF)
            transfer = dec(feature)

        transfer = transfer.clamp(0,1)
        vutils.save_image(transfer.data,'%s/%s_%s.png'%(opt.outf,contentName,styleName),normalize=True,scale_each=True,nrow=opt.batchSize)
        print('Transferred image saved at %s%s_%s.png'%(opt.outf,contentName,styleName))

# make the image editable
        photo = Image.open('%s/%s_%s.png'%(opt.outf,contentName,styleName))
        drawing = ImageDraw.Draw(photo)
        text='@Majid Momeni-Moghaddam             Stylized by: M. Amintoosi '
        pos=(15, 480)
        black = (3, 8, 12)
        font = ImageFont.truetype("C:/Windows/Fonts/BERNHC.ttf", 20)
        drawing.text(pos, text,  font=font)
#         photo.show()
        wtrMrkFileName = '%s/%s_%s.jpg'%(opt.outf,contentName,styleName)
        photo.save(wtrMrkFileName)

d:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\upsampling.py:177: UserWarning: nn.UpsamplingNearest2d is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.UpsamplingNearest2d is deprecated. Use nn.functional.interpolate instead.")
d:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


Transferred image saved at ArtisticMMMphotoes/fox1_018.png
Transferred image saved at ArtisticMMMphotoes/fox1_088.png
Transferred image saved at ArtisticMMMphotoes/fox1_10.png
Transferred image saved at ArtisticMMMphotoes/fox1_100014.png
Transferred image saved at ArtisticMMMphotoes/fox1_100024.png
Transferred image saved at ArtisticMMMphotoes/fox1_100028.png
Transferred image saved at ArtisticMMMphotoes/fox1_100122.png
Transferred image saved at ArtisticMMMphotoes/fox1_10022.png
Transferred image saved at ArtisticMMMphotoes/fox1_1130.png
Transferred image saved at ArtisticMMMphotoes/fox1_117.png
Transferred image saved at ArtisticMMMphotoes/fox1_14.png
Transferred image saved at ArtisticMMMphotoes/fox1_1418.png
Transferred image saved at ArtisticMMMphotoes/fox1_15.png
Transferred image saved at ArtisticMMMphotoes/fox1_1538.png
Transferred image saved at ArtisticMMMphotoes/fox1_261.png
Transferred image saved at ArtisticMMMphotoes/fox1_27.png
Transferred image saved at ArtisticMMMphoto

In [ ]:
for ci,(content,contentName) in enumerate(content_loader):
    contentName = contentName[0]
    contentV.data.resize_(content.size()).copy_(content)
    for sj,(style,styleName) in enumerate(style_loader):
        styleName = styleName[0]
        styleV.data.resize_(style.size()).copy_(style)

        # forward
        with torch.no_grad():
            sF = vgg(styleV)
            cF = vgg(contentV)

            if(opt.layer == 'r41'):
                feature,transmatrix = matrix(cF[opt.layer],sF[opt.layer])
            else:
                feature,transmatrix = matrix(cF,sF)
            transfer = dec(feature)

        transfer = transfer.clamp(0,1)
        vutils.save_image(transfer.data,'%s/%s_%s.png'%(opt.outf,contentName,styleName),normalize=True,scale_each=True,nrow=opt.batchSize)
        print('Transferred image saved at %s%s_%s.png'%(opt.outf,contentName,styleName))


In [11]:
ss = '%s/%s_%s.png'%(opt.outf,contentName,styleName)
print(ss)

ArtisticMMMphotoes//fox1_018.png
